## In this example we are going to use the GEMINI Model for both Query and Response generation

In [1]:
#imports 
import os 
import re
import time
import pandas as pd 
import streamlit as st 
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine, text
import google.generativeai as genai

c:\Users\admin\UpForce\Company Projects\UniversalChatBot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Environment variables
#GEMINI api key
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [3]:
# Database connection
db_user = "postgres" #Add your database creditional
db_password = "Password" #Add your database creditional
db_host = "localhost" #Add your database creditional
db_name = "schoolDB" #Add your database creditional
db_port = 5432 #Add your database creditional
SQLALCHEMY_DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
db = SQLDatabase.from_uri(SQLALCHEMY_DATABASE_URL)

In [4]:
#database
db_name = "postgresql"

In [5]:
# Model initialization
#GEMINI Model
gemini_model = genai.GenerativeModel('gemini-pro')

In [6]:
#table information
table_info = """
        -> The database contains the following tables:

    1. Students: Contains information about students
        - StudentID: Unique identifier for the student
        - FirstName: First name of the student
        - LastName: Last name of the student
        - DateOfBirth: Student's birthdate
        - Grade: The grade or class the student is in
        - Email: Student's email address
        - Address: Home address of the student

    2. Teachers: Contains information about teachers
        - TeacherID: Unique identifier for the teacher
        - FirstName: First name of the teacher
        - LastName: Last name of the teacher
        - Email: Teacher's email address
        - Specialty: The subject or area the teacher specializes in
        - PhoneNumber: Teacher's contact number

    3. Courses: Contains information about courses offered
        - CourseID: Unique identifier for the course
        - CourseName: Name of the course
        - Description: A brief description of the course content
        - TeacherID: The identifier for the teacher responsible for the course

    4. Enrollments: Tracks student enrollments in courses
        - EnrollmentID: Unique identifier for the enrollment record
        - StudentID: Identifier for the student
        - CourseID: Identifier for the course
        - EnrollmentDate: The date when the student enrolled in the course
        - Grade: The grade received by the student in the course

    5. Classrooms: Contains information about classrooms
        - ClassroomID: Unique identifier for the classroom
        - ClassroomName: Name or number of the classroom
        - Location: The physical location of the classroom
        - Capacity: How many individuals the classroom can accommodate

    6. Schedule: Organizes when and where courses take place
        - ScheduleID: Unique identifier for the schedule entry
        - CourseID: Identifier for the course
        - ClassroomID: Identifier for the classroom where the course is held
        - StartTime: When the course begins
        - EndTime: When the course ends
        - DaysOfWeek: Which days of the week the course is held on
"""

In [7]:
# Define the prompt to be passed to the model
prompt = f"""
    -> If the input is a greeting, respond appropriately with a greeting message, return only greeting message for any greeting input.\n
    -> Do not provide any query for update and deletion related tasks.
    -> You are an expert in translating English questions into detailed SQL queries for {db_name} database. Your task is to create a robust SQL query based on the user input using given table information.
    -> If the user question is not suitable for a SQL query, then respond appropriately as per the question.
    -> Ensure the use of single quotes ('') throughout the query, excluding double quotes. Use three backticks (```) at the beginning and end of the query, do not use \\n in the query.
    -> always limit the query to retrieve information for only 10 datapoints each time.\n
    -> For date-related operations, use the 'EXTRACT' function.\n
    -> The database contains the following tables; focus on the relevant table for the given information. Use the following information to form the query:\n
    
    {table_info}
"""

In [8]:
# input Prompt Generator using User Input
def generate_prompt(question, prompt):
    pt = f"""
        **User Question:**
        {question}
        **Additional Information:**
        {prompt}
    """
    return pt

In [9]:
# This function retrieves a SQL query from the model output.
def get_query(output):
    
    # Define a regular expression pattern to find the SQL query enclosed between ```sql and ```
    pattern = re.compile(r"```sql\n(.*?)\n```", re.DOTALL)

    # Search for the pattern in the output string
    match = pattern.search(output)

    # Check if a match is found
    if match:
        # Extract the SQL query
        sql_query = f'{match.group(1).strip()}'
        return sql_query
    else:
        # Inform if no query is found
        print('Query is not found.')

In [10]:
# This function fetches data from the database using a specified SQL query.
def get_data(query):
    # Create a database engine
    engine = create_engine(SQLALCHEMY_DATABASE_URL)

    try:
        # Use a context manager to handle connections and transactions automatically
        with engine.connect() as connection:
            # Convert the query into a format suitable for execution
            query = text(query)

            # Execute the query
            result = connection.execute(query)

            # Fetch all the rows returned by the query
            rows = result.fetchall()
            return rows

    except Exception as e:
        # Print an error message if an exception occurs
        print(f"Error: {e}")

In [11]:
# This function generates a final prompt for an expert answer generator based on the provided inputs.
def generate_final_prompt(question, sql, data, tables):
    final_prompt_template = f"""
        ==> You are an expert answer generator. Your task is to form a comprehensive answer from the given data.\n
        ==> If the data is not available, provide an answer that the data is not available in your language.\n
        ==> If the user input is greeting then greet the user.\n
        ==> Do not return any SQL query or any irrelevant data.\n
        ==> You do not have permission to Delete or Update any table, if user ask to do it reply approprietly that you do not have permission to update database.\n
        **User Question:**
        {question}

        **SQL Query:**
        {sql}

        **Table information:**
        {tables}

        **Data:**
        {data}

    Note: 
        ==> Your task is to generate an appropriate answer based on the given question using the provided data. Consider the SQL Query as a reference.\n
        ==> All commam seperated data should be presented in tabular format only 
        ==> If you are unable to find a relevant answer, please state that there is insufficient data available.\n

        ### Please craft a well-formed response.
    """
    return final_prompt_template

In [12]:
# This function extracts text content from a response object, specifically designed for GEMINI model output.
def get_text_content(response):
    """
    Extract text content from a response object.
    Parameters:
    - response: The response object containing text content.
    Returns:
    - text_content: Extracted text content.
    """
    if len(response.parts) == 1 and "text" in response.parts[0]:
        # Access the text content if it's a simple text response
        text_content = response.parts[0].text
    else:
        # Handle the case where the response is not a simple text
        # Loop through parts and concatenate text from each part
        text_content = ""
        for part in response.parts:
            if "text" in part:
                text_content += part.text

    return text_content

In [13]:
#user input
user_query = "list down 5 student name"
user_query

'list down 5 student name'

In [14]:
#Generating prompt to create the sql query 
full_query = generate_prompt(question=user_query,prompt=prompt)
print(full_query)


        **User Question:**
        list down 5 student name
        **Additional Information:**
        
    -> If the input is a greeting, respond appropriately with a greeting message, return only greeting message for any greeting input.

    -> Do not provide any query for update and deletion related tasks.
    -> You are an expert in translating English questions into detailed SQL queries for postgresql database. Your task is to create a robust SQL query based on the user input using given table information.
    -> If the user question is not suitable for a SQL query, then respond appropriately as per the question.
    -> Ensure the use of single quotes ('') throughout the query, excluding double quotes. Use three backticks (```) at the beginning and end of the query, do not use \n in the query.
    -> always limit the query to retrieve information for only 10 datapoints each time.

    -> For date-related operations, use the 'EXTRACT' function.

    -> The database contains the f

# GEMINI Model

In [15]:
#Creating sql query
model_response = gemini_model.generate_content(full_query)
response = get_text_content(model_response)
print(response)

```sql
SELECT
  FirstName,
  LastName
FROM Students
LIMIT 5;
```


In [16]:
#Extracting query from response
real_query = get_query(response)
print(real_query)

SELECT
  FirstName,
  LastName
FROM Students
LIMIT 5;


In [17]:
# #Query Performer
final_output = get_data(real_query)

## Displaying the data retrieved from database
print(f"data retrieved from query: {final_output}")

data retrieved from query: [('Daniel', 'Sanchez'), ('Anthony', 'Hughes'), ('Jody', 'Davidson'), ('Kathryn', 'Rodriguez'), ('Paul', 'Miller')]


In [18]:
#generating prompt for final answer generation 
full_prompt = generate_final_prompt(question=user_query,sql=real_query,data=final_output,tables=table_info)

print(full_prompt)


        ==> You are an expert answer generator. Your task is to form a comprehensive answer from the given data.

        ==> If the data is not available, provide an answer that the data is not available in your language.

        ==> If the user input is greeting then greet the user.

        ==> Do not return any SQL query or any irrelevant data.

        ==> You do not have permission to Delete or Update any table, if user ask to do it reply approprietly that you do not have permission to update database.

        **User Question:**
        list down 5 student name

        **SQL Query:**
        SELECT
  FirstName,
  LastName
FROM Students
LIMIT 5;

        **Table information:**
        
        -> The database contains the following tables:

    1. Students: Contains information about students
        - StudentID: Unique identifier for the student
        - FirstName: First name of the student
        - LastName: Last name of the student
        - DateOfBirth: Student's birthda

In [19]:
#answer using openai model
answer = gemini_model.generate_content(full_prompt)
final_answer = get_text_content(answer)
print(final_answer)

| FirstName | LastName |
| ----------- | ----------- |
| Daniel | Sanchez |
| Anthony | Hughes |
| Jody | Davidson |
| Kathryn | Rodriguez |
| Paul | Miller |
